In [1]:
#Imports!

from qiskit.algorithms import VQE, QAOA, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SLSQP, SPSA
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.circuit.library import TwoLocal, EfficientSU2
from qiskit import Aer
# from qiskit.utils import 
# from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_finance.applications.optimization import PortfolioOptimization
# from qiskit_finance import QiskitFinanceError
from qiskit_finance.data_providers import WikipediaDataProvider
from qiskit.utils import algorithm_globals, QuantumInstance
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
%matplotlib widget

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# This function imports stock prices using Quandl and Wikipedia. You can edit the start and end datetimes to consider stock prices over a different time period!
# Data is only availble until April 2018, so use time periods earlier than that.
def import_stock_prices(stocks):
    quandl_token = "HRXZqqxBka_9huabBgoW" 
    wiki = WikipediaDataProvider(
                        token = quandl_token,
                        tickers = stocks,
                        start = datetime.datetime(2016,1,1),
                        end = datetime.datetime(2017,1,1))
    wiki.run()
    return wiki

In [3]:
stocks = [ "AAPL", "NFLX", "AMZN", "IBM", "GOOG"] # NASDAQ tickers. If you want to consider more stocks, look up their tickers and add them to the list!
# Other stock tickers you can add include F, NVDA, GME, etc. 
stock_prices = import_stock_prices(stocks)  # Using one of the helper functions defined above to import stock prices

In [4]:
# This function uses matplotlib to plot normalized stock prices
def plot_stock_prices(stock_prices, form="norm"):
    fig, ax = plt.subplots(figsize=(8,6))
    if stock_prices._data:
        for (cnt, s) in enumerate(stocks):
            start = stock_prices._data[cnt][0]
            eq = stock_prices._data[cnt] / start
            min_ = np.min(stock_prices._data[cnt])
            max_ = np.max(stock_prices._data[cnt])
            norm = (stock_prices._data[cnt] - min_)/(max_- min_)
            if form == "norm": ax.plot(norm, label=s)
            elif form == "eq": ax.plot(eq, label=s)
            elif form == "raw": ax.plot(stock_prices._data[cnt], label=s)
            else: raise ValueError("Invalid 'form' value")
        ax.legend()
        key = {"norm":"Normalised", "raw":"Raw", "eq":"Equalized"}
        ax.set_title(f"{key[form]} Yearly Evolution of Stock Prices")
        plt.xticks(rotation=90)
        plt.tight_layout()
        plt.show()
    else:
        print('No wiki data loaded.')

In [5]:
plot_stock_prices(stock_prices, form="raw")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Traceback (most recent call last):
  Input In [5] in <cell line: 1>
    plot_stock_prices(stock_prices, form="raw")
  Input In [4] in plot_stock_prices
    plt.tight_layout()
  File /opt/conda/lib/python3.8/site-packages/matplotlib/pyplot.py:2302 in tight_layout
    return gcf().tight_layout(pad=pad, h_pad=h_pad, w_pad=w_pad, rect=rect)
  File /opt/conda/lib/python3.8/site-packages/matplotlib/figure.py:3195 in tight_layout
    renderer = _get_renderer(self)
  File /opt/conda/lib/python3.8/site-packages/matplotlib/backend_bases.py:1544 in _get_renderer
    print_method(io.BytesIO())
  File /opt/conda/lib/python3.8/site-packages/matplotlib/backend_bases.py:1648 in wrapper
    return func(*args, **kwargs)
  File /opt/conda/lib/python3.8/site-packages/matplotlib/_api/deprecation.py:412 in wrapper
    return func(*inner_args, **inner_kwargs)
  File /opt/conda/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py:540 in print_png
    FigureCanvasAgg.draw(self)
  File /opt/conda/lib/

In [6]:
plot_stock_prices(stock_prices, form="eq")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Traceback (most recent call last):
  Input In [6] in <cell line: 1>
    plot_stock_prices(stock_prices, form="eq")
  Input In [4] in plot_stock_prices
    plt.tight_layout()
  File /opt/conda/lib/python3.8/site-packages/matplotlib/pyplot.py:2302 in tight_layout
    return gcf().tight_layout(pad=pad, h_pad=h_pad, w_pad=w_pad, rect=rect)
  File /opt/conda/lib/python3.8/site-packages/matplotlib/figure.py:3195 in tight_layout
    renderer = _get_renderer(self)
  File /opt/conda/lib/python3.8/site-packages/matplotlib/backend_bases.py:1544 in _get_renderer
    print_method(io.BytesIO())
  File /opt/conda/lib/python3.8/site-packages/matplotlib/backend_bases.py:1648 in wrapper
    return func(*args, **kwargs)
  File /opt/conda/lib/python3.8/site-packages/matplotlib/_api/deprecation.py:412 in wrapper
    return func(*inner_args, **inner_kwargs)
  File /opt/conda/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py:540 in print_png
    FigureCanvasAgg.draw(self)
  File /opt/conda/lib/p

In [7]:
#num of assets (length of our NASDAQ issues array)
num_assets = len(stocks)

# set budget - how many stocks do you want to buy?
budget = 2

# Defining the portfolio optimization problem
portfolio = PortfolioOptimization(expected_returns=stock_prices.get_period_return_mean_vector(),
                                  covariances=stock_prices.get_period_return_covariance_matrix(),
                                  risk_factor=0.2, budget=budget)

# This creates a quadratic program, which is the form in which the problem can be directly
# passed to Qiskit's solvers
qp = portfolio.to_quadratic_program()

In [8]:
# This Function displays teh result of a VQE or Classical EigenSolver
def display_values(vqe_result, stocks):
    lines = []
    for result in vqe_result.samples[:5]:
            qubits, value, probability = result.x, result.fval, result.probability
            q_str = str(qubits.astype(int)).ljust(16)
            arr = np.array(qubits)
            opt = [str(stock[0])+' ' for stock in np.take(stocks, np.argwhere(arr))]
            opt_str = "".join(opt).ljust(16)
            lines.append((opt_str, f'{q_str} {opt_str} {round(value,4)} \t {round(probability,4)}'))

    print(f' Optimal Stock Combination: {lines[0][0]}')
    print('\n------------------------ Top Results ------------------------')
    print('qubits \t\t selection \t value \t\t probability')
    print('--------------------------------------------------------------')
    [print(p[1]) for p in lines]
    print('--------------------------------------------------------------')

# This function draws the value of the portfolio against other the stocks
def plot_portfolio_against_market(result, stock_prices):
    portfolio_sel = np.argwhere(result.samples[0].x).reshape(-1)
    fig, ax = plt.subplots(figsize=(8,6))
    if stock_prices._data:
        portfolio_sum = []
        for (cnt, s) in enumerate(stocks):
            start = stock_prices._data[cnt][0]
            eq = stock_prices._data[cnt] / start
            if cnt in portfolio_sel:
                portfolio_sum.append(eq)
                ax.plot(eq, label=f"{s} (Selected)", linestyle="--", alpha=0.5)
            else:
                ax.plot(eq, label=s, alpha=0.15)
        sab = sum(portfolio_sum)
        # This has no purpose besides shifting the color because I don't like brown
        next(ax._get_lines.prop_cycler)   
        ax.plot(sab/2, label="Portfolio Value")
        ax.set_title("Portfolio Performance") 
        ax.legend()
        plt.xticks(rotation=90)
        plt.tight_layout()
        plt.show()

    else:
        print('No wiki data loaded.')

In [9]:
exact_mes = NumPyMinimumEigensolver() # Specifying the classical solver we want to use - NumPyMinimumEigensolver
exact_eigensolver = MinimumEigenOptimizer(exact_mes) 
result_classical = exact_eigensolver.solve(qp) # Solving the problem using the classical solver

In [10]:
display_values(result_classical, stocks)
plot_portfolio_against_market(result_classical, stock_prices)

 Optimal Stock Combination: AMZN IBM        

------------------------ Top Results ------------------------
qubits 		 selection 	 value 		 probability
--------------------------------------------------------------
[0 0 1 1 0]      AMZN IBM         -0.0017 	 1.0
--------------------------------------------------------------


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Traceback (most recent call last):
  Input In [10] in <cell line: 2>
    plot_portfolio_against_market(result_classical, stock_prices)
  Input In [8] in plot_portfolio_against_market
    plt.tight_layout()
  File /opt/conda/lib/python3.8/site-packages/matplotlib/pyplot.py:2302 in tight_layout
    return gcf().tight_layout(pad=pad, h_pad=h_pad, w_pad=w_pad, rect=rect)
  File /opt/conda/lib/python3.8/site-packages/matplotlib/figure.py:3195 in tight_layout
    renderer = _get_renderer(self)
  File /opt/conda/lib/python3.8/site-packages/matplotlib/backend_bases.py:1544 in _get_renderer
    print_method(io.BytesIO())
  File /opt/conda/lib/python3.8/site-packages/matplotlib/backend_bases.py:1648 in wrapper
    return func(*args, **kwargs)
  File /opt/conda/lib/python3.8/site-packages/matplotlib/_api/deprecation.py:412 in wrapper
    return func(*inner_args, **inner_kwargs)
  File /opt/conda/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py:540 in print_png
    FigureCanvasAgg.dr